# KNN UYGULAMA

In [1]:
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

## 1. Exploratory Data Analysis

In [2]:
df = pd.read_csv(R"C:\Users\Oguz\Desktop\DCMLBC06\yaz_kampı\diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.shape

(768, 9)

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [6]:
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

## 2. Data Preprocessing & Feature Engineering

In [7]:
y = df["Outcome"]

In [8]:
X = df.drop(["Outcome"], axis=1)

uzaklık temelli ve gradient descent temelli yöntemlerde standartlaştırma yaparak daha hızlı ve doğru sonuçlar elde edebiliriz

In [9]:
X_scaled = StandardScaler().fit_transform(X)

In [37]:
X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

numpy arrayi olarak gelen standartlaştırılmış X değerlerinin sütun isimlerine ihtiyacımız var

In [11]:
X = pd.DataFrame(X_scaled, columns=X.columns)

## 3. Modeling & Prediction

In [12]:
knn_model = KNeighborsClassifier().fit(X, y)

In [13]:
random_user = X.sample(1, random_state=45)

In [38]:
random_user

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
46,-0.844885,0.78573,-0.677523,-1.288212,-0.692891,-0.290972,0.278225,-0.360847


In [14]:
knn_model.predict(random_user)

array([1], dtype=int64)

## 4. Model Evaluation

In [15]:
# kurulan model kullanılarak tüm gözlem birimleri için tahmin edilen değerler
# Confusion matrix için y_pred:
y_pred = knn_model.predict(X)  

In [16]:
# AUC için y_prob:
y_prob = knn_model.predict_proba(X)[:, 1]  #1 sınıfına ait olma olasılıkları

In [39]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



-Precision : 1 olarah tahmin ettiklerimin başarısı
    
-Recall    : Gerçekte 1 olanları 1 olarak tahmin etme başarımız
    
-F1 Skore  : Precision ve Recall un harmonik ortalamasıdır.
    
-Macro AVG : Aritmetik ort

-Weighted avg : Ağırlıklı ort

In [18]:
roc_auc_score(y, y_prob)

0.9017686567164179

Model Doğrulama

In [19]:
cv_results = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

In [20]:
cv_results['test_accuracy'].mean()

0.733112638994992

In [21]:
cv_results['test_f1'].mean()

0.5905780011534191

In [22]:
cv_results['test_roc_auc'].mean()

0.7805279524807827

In [23]:
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

## 5. Hyperparameter Optimization

Parametre: Modellerin veri içerisinden öğrendiği ağırlıklardır.
Hiperparamatre : Kullanıcı tarafından tanımlanması gereken veri seti içerisinden öğrenilemeyen parametrelerdir.

In [25]:
knn_model = KNeighborsClassifier()

In [26]:
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [27]:
knn_params = {"n_neighbors": range(2, 50)}

In [28]:
knn_gs_best = GridSearchCV(knn_model,
                           knn_params,
                           cv=5,
                           n_jobs=-1,
                           verbose=1).fit(X, y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [29]:
knn_gs_best.best_params_

{'n_neighbors': 17}

## 6. Final Model

In [30]:
knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X, y)

In [31]:
cv_results = cross_validate(knn_final,
                            X,
                            y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])

In [32]:
cv_results['test_accuracy'].mean()

0.7669892199303965

In [33]:
cv_results['test_f1'].mean()

0.6170909049720137

In [34]:
cv_results['test_roc_auc'].mean()

0.8127938504542278

In [35]:
random_user = X.sample(1)

In [36]:
knn_final.predict(random_user)

array([0], dtype=int64)